In [ ]:
# 내 구글 드라이브에 연동
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# cmake install
!pip install cmake

In [ ]:
# dlib install
!pip install dlib

In [ ]:
# face_recognition install
!pip install face_recognition

     |████████████████████████████████| 100.2MB 75kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=e8c7a2caffbc2dea30a5c08c3dd7051841128a81e6e8df02a8952a86068a3c9b
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
# 필요한 패키지와 모듈을 불러옴
import cv2
import face_recognition
import pickle
import time

In [ ]:
file_name = 'gdrive/My Drive/CV/Face Recognition/chun_video2.mp4'       # 원본 동영상
encoding_file = 'gdrive/My Drive/CV/Face Recognition/encodings.pickle'  # 저장해 두었던 특성 파일
unknown_name = 'Unknown'  # 이름을 모를 경우 Unknown 으로 지정
# Either cnn  or hog. The CNN method is more accurate but slower. HOG is faster but less accurate.
model_method = 'cnn'
output_name = 'output_chun_video2.mp4'   # Detection 된 output 동영상

In [ ]:
def detectAndDisplay(image):
    start_time = time.time()
    # 이미지를 face_recognition 에서 처리하기 위해 RGB 형식으로 변경
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 얼굴을 detection 하고 boxing 한다.
    boxes = face_recognition.face_locations(rgb, model=model_method)
    # 얼굴 영역(boxing Area)만 encoding(특성값)을 계산한다.
    encodings = face_recognition.face_encodings(rgb, boxes)

    # detection 된 얼굴의 이름를 저장할 배열
    names = []

    for encoding in encodings:
        # boxing 된 특성이 pickle 파일의 encodings(특성값) 과 매칭되는 것이 있는지 찾아낸다.
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        name = unknown_name   # 'Unknown'으로 초기화

        # check to see if we have found a match
        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]  # 매칭된 pickle 파일의 True = encodings(특성값) 인덱스 번호를 찾아온다.
            counts = {}    # 매칭된 얼굴별로 이름과 겹친 Count 수 를 Dictionary 타입으로 저장한다.  

            for i in matchedIdxs:
                name = data["names"][i]   # 매칭된 True = encodings(특성값) 인텍스 번호의 "names" value 를 찾아온다.
                counts[name] = counts.get(name, 0) + 1  # counts dict의 "name" 키값의 value 에 count 누적

            name = max(counts, key=counts.get)   # counts dict에서 key값(이름)들 중에서 가장 value 값(카운팅)이 큰 key값(이름)을 가져옴

        # names 배열에 이름을 추가
        names.append(name)

    # dection 된 얼굴의 boxing 위치 좌표와 이름 매칭
    for ((top, right, bottom, left), name) in zip(boxes, names):
        color = (0, 255, 0)   # boxing color
        line = 2              # boxing line 두께  
        if(name == unknown_name):  # 만약 unknown 이라면 
            color = (0, 0, 255)  
            line = 1
            name = ''   # name 표시 안함      
            
        cv2.rectangle(image, (left, top), (right, bottom), color, line)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, line)

    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))
    
    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                (image.shape[1], image.shape[0]), True)

    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(image)

In [ ]:
# 저장해 두었던 encodings(특성값) 파일을 불러온다.
data = pickle.loads(open(encoding_file, "rb").read())

In [ ]:
# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        break
    detectAndDisplay(frame)

=== A frame took 0.177 seconds
=== A frame took 0.171 seconds
=== A frame took 0.180 seconds
=== A frame took 0.171 seconds
=== A frame took 0.172 seconds
=== A frame took 0.172 seconds
=== A frame took 0.172 seconds
=== A frame took 0.181 seconds
=== A frame took 0.173 seconds
=== A frame took 0.172 seconds
=== A frame took 0.172 seconds
=== A frame took 0.170 seconds
=== A frame took 0.174 seconds
=== A frame took 0.177 seconds
=== A frame took 0.172 seconds
=== A frame took 0.171 seconds
=== A frame took 0.172 seconds
=== A frame took 0.173 seconds
=== A frame took 0.178 seconds
=== A frame took 0.175 seconds
=== A frame took 0.172 seconds
=== A frame took 0.171 seconds
=== A frame took 0.172 seconds
=== A frame took 0.179 seconds
=== A frame took 0.171 seconds
=== A frame took 0.173 seconds
=== A frame took 0.172 seconds
=== A frame took 0.170 seconds
=== A frame took 0.173 seconds
=== A frame took 0.173 seconds
=== A frame took 0.172 seconds
=== A frame took 0.170 seconds
=== A fr